In [1]:
import yaml
import numpy as np

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import directed_hausdorff

from data.dataset import NUFTDataset, PolygonDataset
from train.nuft_trainer import Trainer
from model.nn import build_model
from torchinfo import summary

In [2]:
with open('cfg/nuft.yaml', 'r') as f:
    cfg = yaml.safe_load(f)  
    
import pandas as pd
glyph_df = pd.read_pickle(cfg['test'])
glyph_df_o = glyph_df[glyph_df.trans == 'o'].reset_index(drop=True)
glyph_df_r = glyph_df[glyph_df.trans == 'r'].reset_index(drop=True)
glyph_df_sk = glyph_df[glyph_df.trans == 'sk'].reset_index(drop=True)
glyph_df_sc = glyph_df[glyph_df.trans == 'sc'].reset_index(drop=True)

glyph_set_o = NUFTDataset(glyph_df_o).to_data_list()
glyph_set_r = NUFTDataset(glyph_df_r).to_data_list()
glyph_set_sk = NUFTDataset(glyph_df_sk).to_data_list()
glyph_set_sc = NUFTDataset(glyph_df_sc).to_data_list()

glyph_set_o_poly = PolygonDataset(glyph_df_o)
glyph_set_r_poly = PolygonDataset(glyph_df_r)
glyph_set_sk_poly = PolygonDataset(glyph_df_sk)
glyph_set_sc_poly = PolygonDataset(glyph_df_sc)

osm_df = pd.read_pickle(cfg['osm'])
osm_df_o = osm_df[osm_df.trans == 'o'].reset_index(drop=True)    
osm_df_r = osm_df[osm_df.trans == 'r'].reset_index(drop=True)  

osm_set_o = NUFTDataset(osm_df_o).to_data_list()
osm_set_r = NUFTDataset(osm_df_r).to_data_list()

osm_set_o_poly = PolygonDataset(osm_df_o)
osm_set_r_poly = PolygonDataset(osm_df_r)

melb_df = pd.read_pickle(cfg['melb'])
melb_set = NUFTDataset(melb_df).to_data_list()
melb_set_poly = PolygonDataset(melb_df)

  0%|          | 0/12509 [00:00<?, ?it/s]/data/anaconda3/envs/gdl/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 37139/37139 [00:53<00:00, 694.37it/s]


In [3]:
backbone = ['nuft']
ckpt = 'epoch100'

datasets = {'glyph_set_o':glyph_set_o,
            'glyph_set_r':glyph_set_r,
            'glyph_set_sk': glyph_set_sk,
            'glyph_set_sc': glyph_set_sc,
            'osm_set_o': osm_set_o, 
            'osm_set_r': osm_set_r,
            'melb_set': melb_set}

poly_datasets = {'glyph_set_o': glyph_set_o_poly,
                 'glyph_set_r': glyph_set_r_poly,
                 'glyph_set_sk': glyph_set_sk_poly,
                 'glyph_set_sc': glyph_set_sc_poly,
                 'osm_set_o': osm_set_o_poly, 
                 'osm_set_r': osm_set_r_poly,
                 'melb_set': melb_set_poly}

In [4]:

cfg['nn'] = 'nuft'
cfg['path'] = f'save/{cfg["nn"]}'

model = build_model(cfg=cfg)
trainer = Trainer(cfg=cfg) 
model = trainer.load_ckpt(model, 'epoch100')
model.eval()
print(summary(model))

metric = {}

for key, dataset in datasets.items():
    zs = []
    for data in dataset:
        data.to('cuda')
        z = model.encoder(data.x, data.batch)
        zs.append(z.mean(0).detach().cpu().numpy())

    zs = np.stack(zs, axis=0)
    zs = zs[~np.isnan(zs).any(axis=1)]
    
    # NN neighbors
    nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
    dist, idx = nbrs.kneighbors(zs)
    # Hausdorff dist    
    query_id = idx[:, 0]
    target_id = idx[:, 1:]
    haus_dist = []
    
    #Polygon Dataset
    poly_dataset = poly_datasets[key]
    
    for id, tids in enumerate(target_id):
        qid = query_id[id]
        tem_dist = []
        for tid in tids: 
            dist = directed_hausdorff(poly_dataset[qid].pos.cpu().numpy(), 
                                      poly_dataset[tid].pos.cpu().numpy())
            tem_dist.append(dist[0])
        haus_dist.append(tem_dist)
    
    metric[key] = {'avg' : np.stack(haus_dist, axis=0).mean(axis=0).tolist(), 
                   'std' : np.stack(haus_dist, axis=0).std(axis=0).tolist()}
    
nuft = {'nuft': metric}
nuft

Layer (type:depth-idx)                   Param #
NUFTAE                                   --
├─MLP: 1-1                               --
│    └─ReLU: 2-1                         --
│    └─ModuleList: 2-2                   --
│    │    └─Linear: 3-1                  18,496
│    │    └─Linear: 3-2                  4,160
│    │    └─Linear: 3-3                  4,160
│    └─ModuleList: 2-3                   --
│    │    └─BatchNorm: 3-4               128
│    │    └─BatchNorm: 3-5               128
├─MLP: 1-2                               --
│    └─ReLU: 2-4                         --
│    └─ModuleList: 2-5                   --
│    │    └─Linear: 3-6                  4,160
│    │    └─Linear: 3-7                  4,160
│    │    └─Linear: 3-8                  18,720
│    └─ModuleList: 2-6                   --
│    │    └─BatchNorm: 3-9               128
│    │    └─BatchNorm: 3-10              128
Total params: 54,368
Trainable params: 54,368
Non-trainable params: 0


{'nuft': {'glyph_set_o': {'avg': [0.30401881020835775,
    0.3388042650259606,
    0.3541700461446339,
    0.365316684916466,
    0.371343264830591,
    0.37947851023896695],
   'std': [0.23037420708025294,
    0.2275932426668255,
    0.225248242715072,
    0.2264060496098526,
    0.22446590538485875,
    0.22374862993345312]},
  'glyph_set_r': {'avg': [0.41739858312458966,
    0.44214029999651044,
    0.4567767984200117,
    0.4630147879435078,
    0.4700732584255546,
    0.47213215002135905],
   'std': [0.23443798407945263,
    0.22997682602673136,
    0.22783971722812865,
    0.2260439918828496,
    0.22428319289709484,
    0.22035718200694251]},
  'glyph_set_sk': {'avg': [0.4655259770041129,
    0.48877774072048286,
    0.4960720314014189,
    0.501790010821753,
    0.5023154470855378,
    0.5059141689363239],
   'std': [0.24536992556989864,
    0.24282900772765822,
    0.23973816161130435,
    0.23707833751679225,
    0.23665074394632832,
    0.23493622619578652]},
  'glyph_set_sc